In [35]:
import pandas as pd
import glob
import nltk
from string import punctuation




tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [36]:
text=open('articles/jrme_test.txt','r').read()

In [37]:
#tokenizer.tokenize(text)

In [38]:
text[:100]

' \n Chapter 1: Making Mathematics Work in School\n\nAuthor(s): Deborah Loewenberg Ball, Jennifer Lewis '

In [39]:
male_words=set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his','boy','boyfriend','boyfriends','boys','brother','brothers','dad','dads','dude','father','fathers','fiance','gentleman','gentlemen','god','grandfather','grandpa','grandson','groom','he','himself','husband','husbands','king','male','man','mr','nephew','nephews','priest','prince','son','sons','uncle','uncles','waiter','widower','widowers'])
female_words=set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','aunt','aunts','bride','daughter','daughters','female','fiancee','girl','girlfriend','girlfriends','girls','goddess','granddaughter','grandma','grandmother','herself','ladies','lady','lady','mom','moms','mother','mothers','mrs','ms','niece','nieces','priestess','princess','queens','she','sister','sisters','waitress','widow','widows','wife','wives','woman'])

In [72]:
def gender_the_sentence(sentence_words):
    mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(female_words.intersection(sentence_words))

    if mw_length>0 and fw_length==0:
        gender='male'
    elif mw_length==0 and fw_length>0: 
        gender='female'
    elif mw_length>0 and fw_length>0: 
        gender='both'
    else:
        gender='none'
    return gender

In [73]:
def is_it_proper(word):
        if word[0]==word[0].upper():
            case='upper'
        else:
            case='lower'
        
        word_lower=word.lower()
        try:
            proper_nouns[word_lower][case] = proper_nouns[word_lower].get(case,0)+1
        
        except Exception:
            #This is triggered when the word hasn't been seen yet
            proper_nouns[word_lower]= {case:1}

In [74]:
def increment_gender(sentence_words,gender):
    sentence_counter[gender]+=1
    word_counter[gender]+=len(sentence_words)
    for word in sentence_words:
        word_freq[gender][word]=word_freq[gender].get(word,0)+1

In [75]:
sexes=['male','female','none','both']
sentence_counter={sex:0 for sex in sexes}
word_counter={sex:0 for sex in sexes}
word_freq={sex:{} for sex in sexes}
proper_nouns={}

In [76]:
file_list = glob.glob('articles/*.txt')
for filename in file_list:
    text=open(filename,'rt', errors = 'ignore').read()
    
    #Split into sentences
    sentences=tokenizer.tokenize(text)
    
    for sentence in sentences:
        #word tokenize and strip punctuation
            sentence_words=sentence.split()
            sentence_words=[w.strip(punctuation) for w in sentence_words 
                            if len(w.strip(punctuation))>0]
            
            #figure out how often each word is capitalized
            [is_it_proper(word) for word in sentence_words[1:]]

            #lower case it
            sentence_words=set([w.lower() for w in sentence_words])
            
            #Figure out if there are gendered words in the sentence by computing the length of the intersection of the sets
            gender=gender_the_sentence(sentence_words)

            #Increment some counters
            increment_gender(sentence_words,gender)

In [77]:
proper_nouns=set([word for word in proper_nouns if  
                  proper_nouns[word].get('upper',0) / 
                  (proper_nouns[word].get('upper',0) + 
                   proper_nouns[word].get('lower',0))>.50])

In [78]:
common_words=set([w for w in sorted (word_freq['female'],
                                     key=word_freq['female'].get,reverse=True)[:1000]]+[w for w in sorted (word_freq['male'],key=word_freq['male'].get,reverse=True)[:1000]])

common_words=list(common_words-male_words-female_words-proper_nouns)

In [79]:
male_percent={word:(word_freq['male'].get(word,0) / word_counter['male']) 
              / (word_freq['female'].get(word,0) / word_counter['female']+word_freq['male'].get(word,0)/word_counter['male']) for word in common_words}

In [80]:
print( '%.1f%% gendered' % (100*(sentence_counter['male']+sentence_counter['female'])/(sentence_counter['male']+sentence_counter['female']+sentence_counter['both']+sentence_counter['none'])))
print( '%s sentences about men.' % sentence_counter['male'])
print( '%s sentences about women.' % sentence_counter['female'])
print( '%.1f sentences about men for each sentence about women.' % (sentence_counter['male']/sentence_counter['female']))

14.4% gendered
207 sentences about men.
471 sentences about women.
0.4 sentences about men for each sentence about women.


In [81]:
header ='Ratio\tMale\tFemale\tWord'
print( 'Male words')
print( header)
for word in sorted (male_percent,key=male_percent.get,reverse=True)[:50]:
    try:
        ratio=male_percent[word]/(1-male_percent[word])
    except:
        ratio=100
    print( '%.1f\t%02d\t%02d\t%s' % (ratio,word_freq['male'].get(word,0),word_freq['female'].get(word,0),word))

print( '\n'*2)
print( 'Female words')
print( header)
for word in sorted (male_percent,key=male_percent.get,reverse=False)[:50]:
    try:
        ratio=(1-male_percent[word])/male_percent[word]
    except:
        ratio=100
    print( '%.1f\t%01d\t%01d\t%s' % (ratio,word_freq['male'].get(word,0),word_freq['female'].get(word,0),word))

Male words
Ratio	Male	Female	Word
100.0	01	00	behavior
100.0	01	00	refrained
100.0	01	00	appreciate
100.0	01	00	scientists
100.0	01	00	incites
100.0	01	00	twos).8
100.0	01	00	interpretation
100.0	02	00	revises
100.0	01	00	unstable
100.0	01	00	popular
100.0	01	00	thinking-a
100.0	01	00	underachievement
100.0	01	00	material
100.0	01	00	couched
100.0	01	00	wise
100.0	01	00	breath
100.0	01	00	term
100.0	02	00	studied
100.0	01	00	insight
100.0	01	00	three-second
100.0	01	00	reorganizes
100.0	01	00	tense
100.0	01	00	corresponding
100.0	02	00	meet
100.0	01	00	are'"2
100.0	01	00	redirecting
100.0	01	00	reply
100.0	03	00	the-1
100.0	01	00	train
100.0	01	00	forego
100.0	02	00	detailed
100.0	01	00	ratifies
100.0	01	00	correct
100.0	01	00	suspect
100.0	01	00	systems
100.0	01	00	allowed
100.0	01	00	surprising
100.0	01	00	up-front
100.0	02	00	don't
100.0	01	00	statement.p
100.0	01	00	archetypical
100.0	01	00	autism
100.0	01	00	designated
100.0	01	00	impressions
100.0	01	00	un
100.0	01	00	partici
100

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [3]:
doc = nlp("You are the worst.  This is the best")

In [6]:
for i in doc.sents:
    print(i)

You are the worst.  
This is the best


In [7]:
for i in doc.sents:
    print(i.sentiment)

0.0
0.0


In [8]:
doc.sentiment

0.0

()